In [300]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [301]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [302]:
pip install nltk

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install rouge-score

In [ ]:
pip install numpy

In [ ]:
pip install pyarrow

In [ ]:
import torch
import warnings
from transformers import AutoTokenizer, AutoModelForCausalLM
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
models = [
    "microsoft/DialoGPT-medium",
    "Pi3141/DialoGPT-medium-elon-2",
    "0xDEADBEA7/DialoGPT-small-rick",
    "satvikag/chatbot",
    "microsoft/DialoGPT-large"
]

In [ ]:
prompts = [
    "Describe your dream vacation destination.",
    "Share a hobby or activity you enjoy in your free time.",
    "What's your favorite type of cuisine, and why?",
    "If you could learn any new skill, what would it be?",
    "Tell me about a memorable achievement or accomplishment in your life."
]

responses = [
    "My dream vacation destination is a tropical island with pristine beaches and crystal-clear waters.",
    "In my free time, I enjoy painting landscapes and exploring my artistic side.",
    "I love Italian cuisine because of its delicious pasta dishes and flavorful sauces.",
    "If I could learn a new skill, I would choose to play a musical instrument, like the piano.",
    "One of my most memorable achievements was completing a marathon and achieving a personal best."
]

In [ ]:
results_dict = {}

In [ ]:
def calculate_bleu(references, candidates):
    smoothie = SmoothingFunction().method4
    return corpus_bleu(references, candidates, smoothing_function=smoothie)

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

def calculate_bleu(references, candidates):
    smoothie = SmoothingFunction().method4
    references_tokenized = [tuple(ref.split()) for sublist in references for ref in sublist]
    candidates_tokenized = [tuple(cand.split()) for sublist in candidates for cand in sublist]
    return corpus_bleu([references_tokenized], [candidates_tokenized], smoothing_function=smoothie)



In [ ]:
def calculate_rouge(references, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(references, candidate)
    rouge1 = scores['rouge1'].fmeasure
    rouge2 = scores['rouge2'].fmeasure
    rougeL = scores['rougeL'].fmeasure
    return rouge1, rouge2, rougeL

In [ ]:
for model_name in models:
    bleu_scores = []
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    response_lengths = []

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(model_name)

    input_ids = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)["input_ids"]

    with torch.no_grad():
        output = model.generate(input_ids=input_ids, max_length=100)

    generated_responses = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output]

    for response, generated_response in zip(responses, generated_responses):
        # Calculate BLEU score
        bleu_score = calculate_bleu([[response]], [[generated_response]])
        bleu_scores.append(bleu_score)

        # Calculate ROUGE scores
        rouge1, rouge2, rougeL = calculate_rouge(response, generated_response)
        rouge1_scores.append(rouge1)
        rouge2_scores.append(rouge2)
        rougeL_scores.append(rougeL)

        # Calculate response length
        response_lengths.append(len(generated_response.split()))

    # Calculate average scores
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
    avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
    avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
    avg_response_length = sum(response_lengths) / len(response_lengths)

    # Store results
    results_dict[model_name] = {
        "BLEU": avg_bleu_score,
        "ROUGE-1": avg_rouge1_score,
        "ROUGE-2": avg_rouge2_score,
        "ROUGE-L": avg_rougeL_score,
        "Response Length": avg_response_length
    }



In [ ]:
results_df = pd.DataFrame(results_dict).T

In [ ]:
results_df.to_csv('results.csv')
data = pd.read_csv('results.csv')

In [ ]:
import numpy as np
data=pd.read_csv("results.csv")
weights = '1,1,1,1,1'
impacts = '+,+,+,+,-'
import sys
# HANDLING EXCEPTIONS
if data.shape[1] < 3:
    raise ValueError("Input file does not contain three or more columns.")
if not data.iloc[:, 1:].apply(np.isreal).all().all():
    raise ValueError("Columns from 2nd to last do not contain numeric values only.")
if len(weights.split(',')) != len(impacts.split(',')) != data.shape[1] - 1:
    raise ValueError("Number of weights, impacts, and columns must be the same.")
if not all(impact in ['+', '-'] for impact in impacts.split(',')):
    raise ValueError("Impacts must be either +ve or -ve.")
    
# TOPSIS PROGRAM
def vector_normalization(data):
    normalized_data = data.iloc[:, 1:].apply(
        lambda x: x / np.sqrt(np.sum(x**2)), axis=0)
    return normalized_data

normalized_data = vector_normalization(data)
weighted_normalized_matrix = normalized_data * list(map(float, weights.split(',')))
ideal_best = weighted_normalized_matrix.max(
) if impacts[0] == '+' else weighted_normalized_matrix.min()
ideal_worst = weighted_normalized_matrix.min(
) if impacts[0] == '+' else weighted_normalized_matrix.max()
performance_score = np.sqrt(np.sum((weighted_normalized_matrix - ideal_worst)**2, axis=1)) / (
    np.sqrt(np.sum((weighted_normalized_matrix - ideal_best)**2, axis=1)) +
    np.sqrt(np.sum((weighted_normalized_matrix - ideal_worst)**2, axis=1))
)

result_topsis = data.copy()
result_topsis['Topsis Score'] =  performance_score
result_topsis['Rank'] = result_topsis['Topsis Score'].rank(ascending=False)

In [ ]:
plt.figure(figsize=(10, 5))
plt.barh(topsis_results_sorted.index, topsis_results_sorted['Topsis Score'], color='green')
plt.xlabel('TOPSIS Score')
plt.ylabel('Model')
plt.title('TOPSIS Ranking of Models')
plt.gca().invert_yaxis() 

In [ ]:
topsis_columns = ['Unnamed: 0', 'BLEU', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'Response Length', 'Topsis Score', 'Rank']

# Create a new DataFrame with the selected columns for TOPSIS results
topsis_csv_data = topsis_results[topsis_columns]

# Save the new TOPSIS results to a CSV file
topsis_csv_data.to_csv('topsis_results.csv', index=False)

print("TOPSIS results saved to topsis_results.csv")

# Plotting TOPSIS Bar Graph
plt.figure(figsize=(10, 5))
plt.barh(result_topsis['Unnamed: 0'], result_topsis['Topsis Score'], color='purple')
plt.xlabel('TOPSIS Score')
plt.ylabel('Model')
plt.title('TOPSIS Ranking of Models')
plt.gca().invert_yaxis() 
plt.tight_layout()
plt.savefig('topsis_BarGraph.png')
plt.show()

print("TOPSIS Bar Graph saved to topsis_BarGraph.png")

# Calculate BLEU score
bleu_score = calculate_bleu([response], [generated_response])

# Plotting function for each metric
metrics = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L','BLEU']

for metric in metrics:
    plt.figure(figsize=(10, 6))
    plt.bar(data.index, data[metric], color='purple')
    plt.xlabel('Model')
    plt.ylabel(metric)
    plt.title(f'{metric} Comparison')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{metric}_comparison.png')
    plt.show()

# Plotting response length
plt.figure(figsize=(10, 6))
plt.bar(data.index, data['Response Length'], color='purple')
plt.xlabel('Model')
plt.ylabel('Response Length')
plt.title('Response Length Comparison')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('response_length_comparison.png')
plt.show()